<a href="https://colab.research.google.com/github/shivangsingh26/FL-BC-BTP/blob/master/fvProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow federated
!pip install gdown

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import zipfile

# Unzip the datasets
zip_path = '/content/drive/My Drive/BTP.zip'
unzip_path = '/content/datasets'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Function to load and preprocess data with augmentation
def load_data(client_path):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        os.path.join(client_path, 'train'),
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

    test_generator = test_datagen.flow_from_directory(
        os.path.join(client_path, 'test'),
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

    return train_generator, test_generator

clients = ['/content/datasets/Non-iid datasets/non_iid_subset_1','/content/datasets/Non-iid datasets/non_iid_subset_2','/content/datasets/Non-iid datasets/non_iid_subset_3','/content/datasets/Non-iid datasets/non_iid_subset_4',]
data_paths = [os.path.join(unzip_path, client) for client in clients]

train_generators = []
test_generators = []
for path in data_paths:
    train_gen, test_gen = load_data(path)
    train_generators.append(train_gen)
    test_generators.append(test_gen)


Found 19012 images belonging to 19 classes.
Found 4748 images belonging to 19 classes.
Found 19185 images belonging to 19 classes.
Found 4791 images belonging to 19 classes.
Found 19703 images belonging to 19 classes.
Found 4921 images belonging to 19 classes.
Found 19530 images belonging to 19 classes.
Found 4878 images belonging to 19 classes.


In [ ]:
def create_model(num_classes=19):
    base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                                   include_top=False,
                                                   weights='imagenet')
    base_model.trainable = False

    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


In [ ]:
import numpy as np

# Define number of communication rounds and hyperparameters grid
num_rounds = 10
hyperparameters_grid = {
    'RR': [0.3, 0.5, 0.7],
    'epochs': [5, 10]
}

# Initialize models for each client
client_models = [create_model(num_classes=19) for _ in range(4)]

def fed_wpr(client_models, RR):
    new_weights_list = []
    client_weights = [model.get_weights() for model in client_models]

    for client_id in range(len(client_models)):
        new_weights = []
        for layer_weights in zip(*client_weights):
            weighted_sum = np.sum([w * RR for w in layer_weights], axis=0)
            personalized_weights = (1 - RR) * layer_weights[client_id] + weighted_sum
            new_weights.append(personalized_weights)
        new_weights_list.append(new_weights)

    return new_weights_list

def train_and_evaluate(client_models, train_generators, test_generators, RR, epochs, num_rounds=10):
    for round_num in range(num_rounds):
        print(f'Round {round_num+1}/{num_rounds}')

        for i in range(4):
            print(f'Training client {i+1}')
            client_models[i].fit(train_generators[i], epochs=epochs, validation_data=test_generators[i])

        new_weights_list = fed_wpr(client_models, RR)

        for i in range(4):
            client_models[i].set_weights(new_weights_list[i])

    avg_accuracy = 0
    for i in range(4):
        loss, accuracy = client_models[i].evaluate(test_generators[i])
        avg_accuracy += accuracy
        print(f'Client {i+1} - Loss: {loss}, Accuracy: {accuracy}')

    avg_accuracy /= 4
    return avg_accuracy

def grid_search(hyperparameters_grid):
    best_accuracy = 0
    best_params = {}

    for RR in hyperparameters_grid['RR']:
        for epochs in hyperparameters_grid['epochs']:
            print(f'Evaluating RR = {RR}, epochs = {epochs}')
            # Reset models for each hyperparameter combination
            client_models = [create_model(num_classes=19) for _ in range(4)]

            avg_accuracy = train_and_evaluate(client_models, train_generators, test_generators, RR, epochs)

            print(f'Average accuracy for RR = {RR}, epochs = {epochs}: {avg_accuracy}')

            if avg_accuracy > best_accuracy:
                best_accuracy = avg_accuracy
                best_params = {'RR': RR, 'epochs': epochs}

    return best_params, best_accuracy

best_params, best_accuracy = grid_search(hyperparameters_grid)
print(f'Best parameters: {best_params} with average accuracy: {best_accuracy}')


Evaluating RR = 0.3, epochs = 5
Round 1/10
Training client 1
Epoch 1/5
595/595 [==============================] - 303s 500ms/step - loss: 1.8780 - accuracy: 0.4173 - val_loss: 1.8219 - val_accuracy: 0.4560
Epoch 2/5
595/595 [==============================] - 261s 439ms/step - loss: 1.3790 - accuracy: 0.5615 - val_loss: 1.2540 - val_accuracy: 0.6017
Epoch 3/5
595/595 [==============================] - 251s 423ms/step - loss: 1.1861 - accuracy: 0.6217 - val_loss: 1.1426 - val_accuracy: 0.6226
Epoch 4/5
595/595 [==============================] - 249s 418ms/step - loss: 1.0655 - accuracy: 0.6553 - val_loss: 1.0503 - val_accuracy: 0.6542
Epoch 5/5
595/595 [==============================] - 251s 422ms/step - loss: 0.9831 - accuracy: 0.6845 - val_loss: 1.0184 - val_accuracy: 0.6691
Training client 2
Epoch 1/5
600/600 [==============================] - 318s 525ms/step - loss: 1.6546 - accuracy: 0.4797 - val_loss: 1.1871 - val_accuracy: 0.6193
Epoch 2/5
600/600 [==============================] 

In [ ]:
for i in range(4):
    loss, accuracy = client_models[i].evaluate(test_generators[i])
    print(f'Client {i+1} - Loss: {loss}, Accuracy: {accuracy}')

149/149 [==============================] - 18s 120ms/step - loss: 594.4900 - accuracy: 0.0545
Client 1 - Loss: 594.489990234375, Accuracy: 0.054549284279346466
150/150 [==============================] - 17s 115ms/step - loss: 1757352.0000 - accuracy: 0.0539
Client 2 - Loss: 1757352.0, Accuracy: 0.05385097116231918
154/154 [==============================] - 21s 138ms/step - loss: 1144.0861 - accuracy: 0.0526
Client 3 - Loss: 1144.0860595703125, Accuracy: 0.05263157933950424
153/153 [==============================] - 10s 66ms/step - loss: 1153.7388 - accuracy: 0.0531
Client 4 - Loss: 1153.73876953125, Accuracy: 0.05309553071856499
